In [1]:
!pip install transformers
!pip install shap
!pip install spicy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import transformers
import shap
import spicy as sp
from transformers import AutoModel, AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from torch.optim import AdamW
from torch.utils import data
from sklearn import metrics
import torch.nn as nn
import pandas as pd
import numpy as np
import argparse
import logging
import random
import torch
import sys
import os

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
PREDICTION_COLUMN = 'Доброжелательность_3'
model_path = '/content/drive/MyDrive/big_5/OCEAN_shap_texts_bert/A_bert_simple_shap.ipynb'

dataset = pd.read_csv('/content/drive/MyDrive/big_5/df_texts_ocean.csv')
dataset['lengths'] = dataset.post_text.apply(lambda x: len(str(x).split()))
dataset = dataset.query('30 > lengths > 15')
data = pd.DataFrame({'text_raw':dataset['post_text'],'text':dataset['post_text_preprocessed'],'target':dataset[PREDICTION_COLUMN]})
#data = data.query('target in [0,2]').sample(5000)
_,data = train_test_split(data, test_size=0.3, stratify=data.target)
data

,text_raw,text,target
171920,ахахахахаахах абассссака SexyBoy :DDDDDDDDSexy...,ахахахахаахах абассссака sexyboy PUNK dddddddd...,1
53052,Праздники прошли и мы снова в строю💪🏻⠀А Вам тя...,праздники прошли и мы снова в строю EMOJI PUNK...,0
233793,"Ореховый, Хлебный или Холщовый Спас.... Хочетс...",ореховый PUNK хлебный или холщовый спас PUNK P...,2
12180,Яркий корт. Яркие гости из Гвинеи.😁🎾Всегда рад...,яркий корт PUNK яркие гости из гвинеи PUNK EMO...,2
179337,"Задание завершено! Мною выполнено задание ""Муж...",задание завершено PUNK мною выполнено задание ...,0
...,...,...,...
103773,"У меня такое впечатление, что при первом запус...",у меня такое впечатление PUNK что при первом з...,0
187474,"Поздравляю со Старым Новым годом!!! Счастья, у...",поздравляю со старым новым годом PUNK PUNK PUN...,1
229107,Доброе утро! 🌷С первым днем весны!!!!Пусть все...,доброе утро PUNK EMOJI с первым днем весны PUN...,2
158944,МегаТест: Будет ли у тебя парень? Тут ты узнае...,мегатест PUNK будет ли у тебя парень PUNK тут ...,1


In [5]:
data.target.value_counts()

1    2819
2    2132
0    1968
Name: target, dtype: int64

In [6]:
len(data)

6919

In [7]:
frames = []
classes = data.target.unique()

for i in classes:
    g = data[data.target == i].sample(1500)
    frames.append(g)

equally_sampled = pd.concat(frames)

In [8]:
equally_sampled

,text_raw,text,target
178847,"Мне нужен ключ, чтобы открыть еще одну комнату...",мне нужен ключ PUNK чтобы открыть еще одну ком...,1
75551,"Любви, уюта, понимания И радостного настроения...",любви PUNK уюта PUNK понимания и радостного на...,1
22214,влюбленная ежиха!!! ххахахахахаха!!! я *пацтол...,влюбленная ежиха PUNK PUNK PUNK ххахахахахаха ...,1
126152,"Lords Mobile ещё круче, чем я себе представлял...",lords mobile ещё круче PUNK чем я себе предста...,1
8995,"Приветики)))Пускай надежды и мечты сбываются, ...",приветики EMOJI PUNK пускай надежды и мечты сб...,1
...,...,...,...
229276,Путешествовать - просто! Мои летние каникулы в...,путешествовать PUNK просто PUNK мои летние кан...,2
233496,"За окном пасмурно и холодно,и ты сидишь пьешь ...",за окном пасмурно и холодно PUNK и ты сидишь п...,2
235872,Да) Мы сговорились)) Чтобы вконец тебя достать...,да PUNK мы сговорились EMOJI PUNK чтобы вконец...,2
98728,"Хочешь сказать,ты где-то у подъезда валяешься ...",хочешь сказать PUNK ты где PUNK то у подъезда ...,2


In [9]:
class Bert_sequence_classifcation(nn.Module):
    def __init__(self, model_name, num_classes):
        super(Bert_sequence_classifcation, self).__init__()
        self.bert = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=num_classes)

    def forward(self, input_ids, attention_mask):
        # Feed input to BERT
        logits = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask, return_dict=True).logits 
        return logits

In [10]:
 # load the model and tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained('xlm-roberta-base')
model = Bert_sequence_classifcation('xlm-roberta-base', 3).to(device)
model.load_state_dict(torch.load(model_path))   #,map_location=torch.device('cpu')))
labels = sorted(model.bert.config.label2id, key=model.bert.config.label2id.get)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

UnpicklingError: ignored

In [ ]:
def f(x):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # print(x)
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=100, truncation=True) for v in x], device=device)
    # print(tv)
    attention_mask = (tv != 0).type(torch.int64)
    # print(attention_mask)
    outputs = model(tv, attention_mask).detach().cpu()
    outputs = torch.log_softmax(outputs, dim=1).numpy()
    # print(outputs)
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores)
    return val

In [ ]:
def predict(x):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # print(x)
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=100, truncation=True) for v in x], device=device)
    # print(tv)
    attention_mask = (tv != 0).type(torch.int64).to(device)
    # print(attention_mask)
    outputs = model(tv, attention_mask).detach().cpu()
    res = torch.log_softmax(outputs, dim=1).argmax(dim=1)
    return res

In [ ]:
predict(['приветик','еще раз привет'])

In [ ]:
data = equally_sampled

In [ ]:
texts = list(data.text)
targets = list(data.target)

def chunkify(texts, targets, n):

  texts_splited = [texts[i::n] for i in range(n)]
  targets_splited = [targets[i::n] for i in range(n)]

  return texts_splited, targets_splited

texts_splited, targets_splited =chunkify(texts, targets, 100)

In [ ]:
len(texts_splited[0])

In [ ]:
from tqdm.auto import tqdm

In [ ]:
predictions = []
for text in tqdm(texts_splited):
  predictions.append(predict(text))

In [ ]:
list_predictions = []
for i in predictions:
  for x in list(i):
    list_predictions.append(int(x))

In [ ]:
len(list_predictions)

In [ ]:
data['predictions'] = list_predictions
data.head()

In [ ]:
polar_people = data.query('target == predictions')
polar_people

In [ ]:
polar_people.target.value_counts()

In [ ]:
polar_people = polar_people.sort_values(by=['target'])
polar_people

In [ ]:
s = 'новым годом с наступающим год лошади'
for i, row in polar_people.iterrows():
  if s in row.text:
    print(row)

In [ ]:
import shap
explainer = shap.Explainer(f, tokenizer, output_names=labels)

In [ ]:
texts = list(polar_people.text)
shap_values = explainer(texts)

In [ ]:
shap.plots.text(shap_values[:10]) # низкий уровень по шкале

In [ ]:
shap.plots.text(shap_values[10:20]) # низкий уровень по шкале

In [ ]:
shap.plots.text(shap_values[20:30]) # низкий уровень по шкале

In [ ]:
shap.plots.text(shap_values[30:40]) # низкий уровень по шкале

In [ ]:
shap.plots.text(shap_values[-10:]) # высокий уровень по шкале

In [ ]:
shap.plots.text(shap_values[-20:-10]) # высокий уровень по шкале

In [ ]:
shap.plots.text(shap_values[-30:-20]) # высокий уровень по шкале

In [ ]:

shap.plots.bar(shap_values[:,:, 2].mean(axis=0), max_display=15,
               order=shap.Explanation.argsort.flip)

In [ ]:

shap.plots.bar(shap_values[:,:, 2].max(axis=0), max_display=15,
               order=shap.Explanation.argsort.flip)

In [ ]:

shap.plots.bar(shap_values[:,:, 0].max(axis=0), max_display=20,
               order=shap.Explanation.argsort.flip)

In [ ]:

shap.plots.bar(shap_values[:,:, 1].max(axis=0), max_display=20,
               order=shap.Explanation.argsort.flip)

In [ ]:
shap.plots.bar(shap_values[:,:,0].mean(0)) # по классу 0

In [ ]:
shap.plots.bar(shap_values[:,:,2].mean(0)) 

In [ ]:
shap.plots.bar(shap_values[:,:,2].max(0)) 

In [ ]:
shap.plots.bar(shap_values[2,:, 2], max_display=15,
               order=shap.Explanation.argsort.flip)

In [ ]:
shap.waterfall_plot(shap_values[2][:, 2], max_display=15)